# Generalized Forum Scraping

### Remarks

In [ ]:
To do:    1. Scrape also when info inside of <class     \>

### Functions

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import nltk
import gensim

C:\Users\Meret\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Be careful to run the cells in order

Functions that find the tags allowing to retrieve the data we need

In [2]:
import re

#Find the tag to scrape the next page
def find_next_tag(x, soup, tag):
    if x != None and x.parent != None:
        next_tag = x.parent.name
        try:
            soup.find_all(next_tag, string="Next")[0]['href'][1:]
            tag[0] = next_tag    #tag.append(next_tag)
            print("The tag for next page is ", next_tag)
        except:
            find_next_tag(x.parent, soup, tag)
        return tag[0]
    else:
        print("tag of the next page is not treated in this program")
        return None

#Help function to find a tag recursively
def find_tags_recursively(x, comp, tags):
    tag = x.name
    try:
        class_name = x['class'][0]
        tags.append(class_name)
        tags.append(tag)
        next_ = x.findNext(tag, {'class': class_name})
    except:
        #print("TAG: ", tag)
        tags.append(tag)
        #print("TAGS: ", tags)
        next_ = x.findNext(tag)
        #print("next: ", next_)
        
    if next_ != None:
        if comp in next_.text:
            print("I've found the right tag, it's ", tags[::-1])
            return tags[::-1]
        else:
            return find_tags_recursively(x.parent, comp, tags)
    else:
        print("Unable to scrape this text from this forum") 

#Find all tags needed to retrieve the information we're looking for (titles of thread, messages, ...)
def find_tags(soup, param1, param2):
    text = soup.find(text=re.compile(param1))
    if text is None:
        print("Unable to scrape this text from this forum")
        return None
    else:
        return find_tags_recursively(text.parent, param2, []) 
    


Functions that scrape the forum thanks to the tags found above

In [129]:
def find_next(soup, next_tag):
    try:
        if next_tag is not None:
            return soup.find_all(next_tag, string="Next")[0]['href'][1:]
        return ""
    except:
        return ""

def collect_forum_data(soup_row, tags, soup_row_ok = 0, msg_or_user="", link=""):
    data = {}
    try:
        if not soup_row_ok:
            soup_row = soup_row.find(tags[len(tags) -1])
    
        data['Title'] = soup_row.text.strip()
        data['Link'] = soup_row['href'][1:]
        
    except:
        pass
    return data

def collect_post_data(soup_row, tags, soup_row_ok, link=""):
    data = {}
    try:        
        data['Link'] = link.replace(PREFIX_URL, '')
        
        if len(tags) == 2 and not soup_row_ok:
            soup_row = soup_row.find_all(tags[len(tags)-1])
        if len(tags) == 3:                       
            soup_row = soup_row.find_all(tags[len(tags)-1])
        elif len(tags) == 4:
            soup_row = soup_row.find(tags[2]).find_all(tags[3])
        
        #concatenate message if it spans over multiple tags (often <p>)
        if soup_row_ok:
            data['Any'] = soup_row.text.strip()
        elif len(soup_row) > 1:
            message = ''
            for msg in soup_row:
                message += msg.text.strip()  
            data['Any'] = message
        else:
            data['Any'] = soup_row[0].text.strip()
          
    except:
        pass
    return data

def collect_recursively(data, soup, tags, next_tag, fcte_name, link="", index=""):
    try:
        if index:
            print(index, end='\r', flush=True)
        
        soup_row_ok = 0
        if len(tags) == 1:
            soup_rows = soup.find_all(tags[0])
            soup_row_ok = 1
        if len(tags) == 2:
            soup_rows = soup.find_all(tags[0], {'class': tags[1]})
            if not soup_rows:
                soup_rows = soup.find_all(tags[0]) 
            else:
                soup_row_ok = 1      
        if len(tags) == 3 or len(tags) == 4:
            soup_rows = soup.find_all(tags[0], {'class': tags[1]})
    
        #print(soup_rows)
        data.extend([fcte_name(soup_row, tags, soup_row_ok, link) for soup_row in soup_rows])
        
        next_url = find_next(soup, next_tag)
        if next_url:
            soup = BeautifulSoup(requests.get(PREFIX_URL + next_url).text, 'html.parser')
            if index:
                return collect_recursively(data, soup, tags, next_tag, fcte_name, link, index+1)
            else:
                return collect_recursively(data, soup, tags, next_tag, fcte_name, link)
        else:
            return data
    except:
        pass
    return data
                
def verify_if_treated(soup, tags):
    if len(tags) > 4 or len(tags) < 1:
        print("This case is not treated yet")
        
    if len(tags) == 3 or len(tags) == 4:
        try:
            soup.find(tags[0], {'class': tags[1]})
        except:
            print("This case is not treated yet")
    
def collect_all_links(soup, tags, next_tag, fcte_name):      #soup or url? Combine (put verifications also to collect)
    verify_if_treated(soup, tags)

    data = collect_recursively([], soup, tags, next_tag, fcte_name)
    return pd.DataFrame(data).dropna()

def collect(forum_df, tags, next_tag, fcte_name):
    data = []
    total = len(forum_df['Link'])
    index = 0
    for url in forum_df['Link']:
        index += 1
        print('{} out of {}'.format(index, total), end='\r', flush=True)
        #url = forum_df['Link'][0]
        soup = BeautifulSoup(requests.get(PREFIX_URL + url).text, 'html.parser')
        verify_if_treated(soup, tags)
        data.extend(collect_recursively([], soup, tags, next_tag, fcte_name, url))
    return pd.DataFrame(data)

### 1. Collect all titles and links of the forum

Enter the 3 parameters needed:

In [166]:
#Holiday Truths
#PREFIX_URL = 'https://www.holidaytruths.co.uk/'
#START_URL = PREFIX_URL + 'forum/america-canada-discussion-forum-f2-0.html'
#title1 = 'ESTA question on employment'
#title2 = 'Vegas Buffets/Restaurants'

#Wrong Planet
#PREFIX_URL = 'http://wrongplanet.net/forums'
#START_URL = PREFIX_URL + '/viewforum.php?f=19'
#title1 = 'RE: Kids w/ Classic Autism, PDD-NOS & Speech Delays'
#title2 = 'Parents on the spectrum'

#Stack Overflow
#PREFIX_URL = 'https://stackoverflow.com/'
#START_URL = PREFIX_URL + 'questions/tagged/forum'
#title1 = 'Should DynamoDB adjacency lists use discrete partition keys to model each type of relationship?'
#title2 = 'How to Bypass [hide] element in Forums?'

#Au Féminin
#PREFIX_URL = 'https://astrologie.aufeminin.com/forum'
#START_URL = PREFIX_URL + '/all'
#title1 = 'coucou....'
#title2 = 'échange serieux en mp'

#Trip Advisor
PREFIX_URL = 'https://www.tripadvisor.co.uk/'
START_URL = PREFIX_URL + 'ShowForum-g1-i12334-Holiday_Travel.html'
title1 = 'See TOP QUESTIONS before posting!'
title2 = 'Use the SEARCH BOX function before posting!'

#Travel Forums (North America Forum)
#PREFIX_URL = 'https://www.travellerspoint.com/'
#START_URL = PREFIX_URL + 'forum.cfm?ForumID=6'
#title1 = 'Travelling to US with a criminal record in the UK'
#title2 = 'Your 5 Favorite North American Trips'

#Forum Maladies
#PREFIX_URL = 'https://forums.maladiesraresinfo.org/'
#START_URL = PREFIX_URL + 'isolement-maladies-rares-f7.html'
#title1 = 'Recherche amitié pour me sortir de'
#title2 = 'Besoin de parler de ma maladie'

In [168]:
soup = BeautifulSoup(requests.get(START_URL).text, 'html.parser')
#print(soup.prettify())

Find the tags to be able to scrape the titles and links and the tag needed to find the next page

In [169]:
right_tags = find_tags(soup, title1, title2)
next_tag = find_next_tag(soup.find(text=re.compile("Next")), soup, [''])
print(next_tag)              #delete this

I've found the right tag, it's  ['b', 'a']
tag of the next page is not treated in this program
None


Scrape all titles and links

In [170]:
threads = collect_all_links(soup, right_tags, next_tag, collect_forum_data)
#print(threads)

In [91]:
threads.to_json('C:/Users/Meret/Documents/EPFL/3Annee/Semestre_5/Projet/Forum_Chatbot/Forum Data/trip_advisor_myscraping.json')

### 2. Collect all usernames, messages of every posts in every link

Enter the parameters needed:

In [171]:
#Holiday Truths
#user1 = 'AnnaM'
#user2 = 'Glynis HT Admin'
#msg1 = 'Hi, I am brand new and hopefully I have put this question in the right area.'
#msg2 = 'd have thought that was fine. Plenty of retired people travel.'

#Wrong Planet
#user1 = 'cyberdad'
#user2 = 'Solvejg'
#msg1 = 'I cope fine in the general parenting area.'
#msg2 = 'Yes.'

#Trip Advisor
user1 = 'BradJill'
user2 = 'Eden7'
msg1 = 'HOW TO USE THE HOLIDAY TRAVEL FORUM!'
msg2 = 'Great advice BradJill.....'

#Travel Forums (North America Forum) enter a sentence from last paragraph!
#user1 = 'jay-london'
#user2 = 'Windmill'
#msg1 = 'Jay'
#msg2 = 'Not judging man, but I'

#Forum Maladies
#threads.Link = threads.Link.apply(lambda row: row.split('ttps://forums.maladiesraresinfo.org/')[1])
#user1 = 'Cindy'
#user2 = 'Modérateur'
#msg1 = "Bonjour c'est Cindy, j'aimerais rencontrer quelqu'un pour amitié"
#msg2 = "Comme nous l'écrivions hier, nous comprenons"
#right_url = "https://forums.maladiesraresinfo.org/recherche-amitie-pour-sortir-isolement-t733.html"

In [172]:
START_URL = PREFIX_URL + threads.Link[0]
soup =  BeautifulSoup(requests.get(START_URL).text, 'html.parser')
#print(soup.prettify())

Check that PREFIX_URL is well defined (optional, run this cell if unable to scrape usernames and messages)

In [173]:
#if START_URL != right_url:
#    print("PREFIX_URL wasn't well defined. Please check it.")

Find the tags to be able to scrape the usernames

In [174]:
user_tags = find_tags(soup, user1, user2)

I've found the right tag, it's  ['div', 'username', 'a', 'span']


Scrape all usernames

In [176]:
user_posts = collect(threads, user_tags, next_tag, collect_post_data)
user_posts.rename(columns={'Any':'Username'}, inplace=True)
#print(user_posts)

Find the tags to be able to scrape the messages (if unable to scrape maybe it's because of multiple <p> in this case enter a sentence of the last paragraph)

In [177]:
message_tags = find_tags(soup, msg1, msg2)

I've found the right tag, it's  ['div', 'postBody', 'p']


Scrape all messages

In [179]:
message_posts = collect(threads, message_tags, next_tag, collect_post_data)
#print(message_posts)

In [ ]:
Sometimes random data is found in the tags of the message (ex. 'p'), this function get rid of this data

In [76]:
def check_messages(data, msg1):
    for i in range(0, len(data)-1):
        if msg1 in data['Any'][i]:
            return i

In [77]:
index = check_messages(message_posts, msg1)
print("index: ", index)
message_posts['Any'] = message_posts['Any'].iloc[index:]
message_posts['Any'] = message_posts['Any'].shift(-index)
#print(message_posts)

index:  0


In [ ]:
print(message_posts)

### 3. Merge the titles, usernames, messages to have all posts

Concatenate the messages with the users

In [182]:
user_posts['Message'] =  pd.Series(message_posts['Any'], index=user_posts.index)
posts= user_posts

In [183]:
posts

,Username,Link,Message
0,BradJill,ShowTopic-g1-i12334-k7867029-See_TOP_QUESTIONS...,HOW TO USE THE HOLIDAY TRAVEL FORUM!It is wort...
1,Eden7,ShowTopic-g1-i12334-k7867029-See_TOP_QUESTIONS...,Great advice BradJill.....any idea how to acce...
2,BradJill,ShowTopic-g1-i12334-k7867029-See_TOP_QUESTIONS...,Now that is a good question. I know this is mu...
3,BradJill,ShowTopic-g1-i12334-k7867031-Use_the_SEARCH_BO...,HOW TO USE THE HOLIDAY TRAVEL FORUM!It is wort...
4,KVE1005,ShowTopic-g1-i12334-k7867031-Use_the_SEARCH_BO...,Very important post and much needed BradJill. ...
5,Eden7,ShowTopic-g1-i12334-k7867031-Use_the_SEARCH_BO...,I agree with you both!
6,BradJill,ShowTopic-g1-i12334-k5876122-How_to_Use_the_Ho...,WELCOME TO THE HOLIDAY FORUM!Have you come to ...
7,Super M,ShowTopic-g1-i12334-k5876122-How_to_Use_the_Ho...,the best Stick can be done for the Holiday Tra...
8,thegoodwitch,ShowTopic-g1-i12334-k5876122-How_to_Use_the_Ho...,"Good post BradjillHowever, I think TA should s..."
9,Eden7,ShowTopic-g1-i12334-k5876122-How_to_Use_the_Ho...,Nicely done BradJill................


Merged with the titles

In [184]:
merged_df = pd.merge(threads, posts, on='Link', how='inner')
merged_df = merged_df.reindex(sorted(merged_df.columns), axis=1)
merged_df.to_json('C:/Users/Meret/Documents/EPFL/3Annee/Semestre_5/Projet/Forum_Chatbot/Forum Data/trip_advisor_threads_myscraping.json')

In [185]:
merged_df

,Link,Message,Title,Username
0,ShowTopic-g1-i12334-k7867029-See_TOP_QUESTIONS...,HOW TO USE THE HOLIDAY TRAVEL FORUM!It is wort...,See TOP QUESTIONS before posting!,BradJill
1,ShowTopic-g1-i12334-k7867029-See_TOP_QUESTIONS...,Great advice BradJill.....any idea how to acce...,See TOP QUESTIONS before posting!,Eden7
2,ShowTopic-g1-i12334-k7867029-See_TOP_QUESTIONS...,Now that is a good question. I know this is mu...,See TOP QUESTIONS before posting!,BradJill
3,ShowTopic-g1-i12334-k7867031-Use_the_SEARCH_BO...,HOW TO USE THE HOLIDAY TRAVEL FORUM!It is wort...,Use the SEARCH BOX function before posting!,BradJill
4,ShowTopic-g1-i12334-k7867031-Use_the_SEARCH_BO...,Very important post and much needed BradJill. ...,Use the SEARCH BOX function before posting!,KVE1005
5,ShowTopic-g1-i12334-k7867031-Use_the_SEARCH_BO...,I agree with you both!,Use the SEARCH BOX function before posting!,Eden7
6,ShowTopic-g1-i12334-k5876122-How_to_Use_the_Ho...,WELCOME TO THE HOLIDAY FORUM!Have you come to ...,How to Use the Holiday Travel Forum,BradJill
7,ShowTopic-g1-i12334-k5876122-How_to_Use_the_Ho...,the best Stick can be done for the Holiday Tra...,How to Use the Holiday Travel Forum,Super M
8,ShowTopic-g1-i12334-k5876122-How_to_Use_the_Ho...,"Good post BradjillHowever, I think TA should s...",How to Use the Holiday Travel Forum,thegoodwitch
9,ShowTopic-g1-i12334-k5876122-How_to_Use_the_Ho...,Nicely done BradJill................,How to Use the Holiday Travel Forum,Eden7


### Comparison with original (get rid when done)

In [ ]:
original_merged_df = pd.read_json('C:/Users/Meret/Documents/EPFL/3Annee/Semestre_5/Projet/Forum_Chatbot/Forum Data/trip_advisor_threads_myscraping.json')

In [ ]:
original_merged_df = original_merged_df.drop(['Replies', 'Timestamp'], axis=1)

In [ ]:
original_merged_df